In [8]:
!pip install --quiet vosk transformers torch gTTS pydub soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.9 MB/s eta 0:00:00:00:0100:01


In [2]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip vosk-model-small-en-us-0.15.zip

--2025-05-23 00:40:00--  https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41205931 (39M) [application/zip]
Saving to: ‘vosk-model-small-en-us-0.15.zip’

vosk-model-small-en 100%[===================>]  39.30M  87.7MB/s    in 0.4s    

2025-05-23 00:40:00 (87.7 MB/s) - ‘vosk-model-small-en-us-0.15.zip’ saved [41205931/41205931]

Archive:  vosk-model-small-en-us-0.15.zip
   creating: vosk-model-small-en-us-0.15/
   creating: vosk-model-small-en-us-0.15/am/
  inflating: vosk-model-small-en-us-0.15/am/final.mdl  
   creating: vosk-model-small-en-us-0.15/graph/
  inflating: vosk-model-small-en-us-0.15/graph/disambig_tid.int  
  inflating: vosk-model-small-en-us-0.15/graph/HCLr.fst  
  inflating: vosk-model-small-en-us-0.15/graph/Gr.fst  
   creating: vosk-mode

In [30]:
from gtts import gTTS
from pydub import AudioSegment

# if you dont want to upload an audio, it generates it for you
QUESTION = "how are you?"
gTTS(QUESTION, lang="en").save("sample_input.mp3")
AudioSegment.from_mp3("sample_input.mp3") \
            .set_frame_rate(16000).set_channels(1) \
            .export("sample_input.wav", format="wav")
print("sample_input.wav generated")

sample_input.wav generated


In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import json, soundfile as sf
from vosk import Model, KaldiRecognizer
from gtts import gTTS
from pydub import AudioSegment
from IPython.display import Audio, display

# 1) Transcribe via VOSK
data, sr = sf.read("sample_input.wav", dtype="int16")
vosk_model = Model("vosk-model-small-en-us-0.15")
rec = KaldiRecognizer(vosk_model, sr)
rec.AcceptWaveform(data.tobytes())
user_text = json.loads(rec.Result()).get("text", "")
print("You said:", user_text)

# 2) Load DialoGPT
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model     = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# 3) Build a prompt with a persona + example
persona  = (
    "The following is a conversation between a friendly, helpful AI assistant and a human.\n"
    "Human: Hello, how are you?\n"
    "Assistant: I'm doing great, thank you for asking! How can I help you today?\n"
)
prompt   = f"{persona}Human: {user_text}\nAssistant:"

# 4) Tokenize & generate
inputs = tokenizer(prompt, return_tensors="pt")
out_ids = model.generate(
    **inputs,
    max_length=inputs.input_ids.shape[-1] + 50,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_p=0.9,
    top_k=50,
    temperature=0.7
)

# 5) Slice off prompt tokens → decode only the assistant’s new words
response_ids = out_ids[0, inputs.input_ids.shape[-1]:]
bot_text     = tokenizer.decode(response_ids, skip_special_tokens=True).strip()
print("DialoGPT reply:", bot_text)

# 6) TTS → reply.wav
gTTS(bot_text, lang="en").save("reply.mp3")
AudioSegment.from_mp3("reply.mp3") \
            .set_frame_rate(16000).set_channels(1) \
            .export("reply.wav", format="wav")

# 7) Play back
print("Question:")
display(Audio("sample_input.wav"))
print("Answer:")
display(Audio("reply.wav"))

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-en-us-0.15/graph/HCLr.fst vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo vosk-model-small-en-us-0.15/graph/phones/word_boundary.int


You said: how are you
DialoGPT reply: How do you know I'm not just a bot?
Question:


Answer:
